In [ ]:
import pandas as pd

pd_siniestros = pd.read_excel('../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx')
excel_path = '../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx'

In [ ]:
###
# 1. Ver todas las hojas
###
print('Hojas disponibles en el archivo:')
hojas = pd.ExcelFile(excel_path).sheet_names
for i, hoja in enumerate(hojas):
    print(f"   {i+1}. {hoja}")

Hojas disponibles en el archivo:
   1. Siniestros
   2. Vehiculos
   3. Actor_vial
   4. Limpieza de datos
   5. Diccionario


In [22]:
###
# 2. Filtrar hojas que no quieres (apuntes, tareas)
###
hojas_interes = []
for hoja in hojas:
    if 'siniestros' in hoja.lower() or 'limpieza' in hoja.lower() or 'diccionario' in hoja.lower():
        continue
    else:
        hojas_interes.append(hoja)

print(f'\nHojas a analizar: {hojas_interes}')


Hojas a analizar: ['Vehiculos', 'Actor_vial']


In [ ]:
###
# 3. Ver todas las columnas de cada hoja de interes
###
def analizar_hoja_detallada(hoja_nombre):
    '''Función para análisis detallado de cada hoja'''
    print(f'\n{'='*60}')
    print(f'Analisis detallado - hoja: {hoja_nombre}')
    print(f'{'='*60}')
    
    # Cargar la hoja
    df_sinistros_clean = pd.read_excel(excel_path, sheet_name=hoja_nombre)
    
    # Información básica
    print(f'Dimensiones: {df_sinistros_clean.shape[0]} filas × {df_sinistros_clean.shape[1]} columnas')
    
    # Columnas
    print(f'\nLista de Columnas ({len(df_sinistros_clean.columns)} total):')
    for i, columna in enumerate(df_sinistros_clean.columns, 1):
        print(f'   {i:2d}. {columna}')
    
    # Tipos de datos
    print(f'\nTipo de dátos:')
    print(df_sinistros_clean.dtypes)
    
    # Valores nulos
    print(f'\nValores nulos (primeras 10 columnas):')
    nulos = df_sinistros_clean.isnull().sum()
    print(nulos.head(10))
    
    return df_sinistros_clean

# Aplicar a cada hoja de interés
dataframes = {}
for hoja in hojas_interes:
    dataframes[hoja] = analizar_hoja_detallada(hoja)


ANÁLISIS DETALLADO - HOJA: Vehiculos
Dimensiones: 608155 filas × 45 columnas

LISTA DE COLUMNAS (45 total):
    1. Codigo_Accidentado
    2. Codigo_Accidente
    3. Formulario
    4. Codigo_Vehiculo
    5. Ccodigo_Victima
    6. FechaAcc
    7. AnnoAcc
    8. mesAcc
    9. DD_Mes_Acc
   10. Dia_Semana_Acc
   11. hourAcc
   12. minuAcc
   13. Localidad
   14. Edad
   15. Sexo
   16. Gravedad_Indicador_Tradicional
   17. Muerte_Posterior
   18. Fecha_CambioGravedad
   19. Gravedad_Indicador_30d
   20. Condicion
   21. Condicion_Especifica
   22. Validación
   23. Tipo_SITP
   24. Con_Bicicleta
   25. Con_Carga
   26. Con_Embriaguez
   27. Con_Huecos
   28. Con_Menores
   29. Con_Moto
   30. Con_Peaton
   31. Con_Persona_Mayor
   32. Con_Rutas
   33. Con_Tpi
   34. Con_Tpp
   35. Con_Velocidad
   36. Con_Sitp
   37. Con_Troncal
   38. Con_Alimentador
   39. Con_Zonal
   40. Con_Provisional
   41. Con_Articulado
   42. Con_Biarticulado
   43. Con_Padron_Dual
   44. Con_Servicio_Especial
 